## Example codes for converting surface and voxel model to different formats

 - The surface conversion codes assume that a directory of obj format surfaces already exsists, which will exsist if using any of the example notebooks, and the code can be run directly from this notebook simply by updating the file paths
 - The voxel conversion codes need to be pasted at the end of the one of the Example notebooks as it relies on having access to a LoopStructural model object.

## Convert geotif of dtm to obj format mesh

In [ ]:
from map2loop.m2l_utils import save_dtm_mesh
obj_path_dir='C:/Users/mark/Desktop/july_models/model2_m'   # directory of existing obj surfaces
dtm_path='C:/Users/mark/Desktop/july_models/model2_m/dtm/' 
save_dtm_mesh(dtm_path,obj_path_dir)


### GeoscienceAnalyst geohy5 format surfaces from directory of obj files

In [ ]:
#code to parse a directory of .obj files and combine them into
#a single geoh5 file that GeoscienceAnalyst can read
#Requires installation of https://github.com/MiraGeoscience/geoh5py 
#(note .geoh5 meshes are 0 based)
#Run this code after obj files have been created (see Example notebooks)

import pandas as pd
from os import listdir
import os
from os.path import isfile, join
from pathlib import Path
import numpy as np
from geoh5py.objects import BlockModel
from geoh5py.workspace import Workspace
from geoh5py.objects import Surface
import omf

def hextofloats(h):
    '''Takes a hex rgb string (e.g. #ffffff) and returns an RGB tuple (float, float, float).'''
    return tuple(int(h[i:i + 2], 16) / 255. for i in (1, 3, 5))  # skip '#'

def geoh5_create_surface_data(obj_path_dir,colour_path):

    h5file_path = obj_path_dir+"/loop.geoh5"

    workspace = Workspace(h5file_path)
    onlyfiles = [f for f in listdir(obj_path_dir) if isfile(join(obj_path_dir, f))]
    colour_index=0
    all_sorts = pd.read_csv(os.path.join(colour_path, 'all_sorts_clean.csv'), ",")
    all_sorts=all_sorts.set_index('code')
    colour_map=open(obj_path_dir+'/loop_colour_map.clr','w')
    colour_map.write('{\tStart\tRed\tGreen\tBlue\t}\n')

    for file in onlyfiles:
        if ('.obj' in file):
            obj=pd.read_csv(obj_path_dir+'/'+file,' ',names=["code","X","Y","Z"])
            indices=obj[obj['code']=='f']
            vertices=obj[obj['code']=='v']
            vertices=vertices.drop(['code'], axis=1)
            indices=indices[list("XYZ")].astype(int)
            i=indices.to_numpy()-1
            v=vertices.to_numpy()
            if(len(i)>0 and len(v)>0):
                # Create a geoh5 surface
                surface = Surface.create(
                          workspace, name=file.replace('.obj',''), vertices=v, cells=i
                            )
                if('Fault_' in file or 'dtm' in file):
                    colours=np.ones(surface.n_cells)*99
                else:
                    colours=np.ones(surface.n_cells)*colour_index
                    rgb=hextofloats(all_sorts.loc[file.replace('.obj','')]['colour'])
                    colour_map.write('{}\t{}\t{}\t{}\n'.format(colour_index,rgb[0],rgb[1],rgb[2]))
                    colour_index=colour_index+1
                    

                surface.add_data({
                    "colour_index": {
                        "association":"CELL",
                        "values": colours
                    }
                })
                
                workspace.save_entity(surface)
                workspace.finalize()
                
    colour_map.write('{}\t{}\t{}\t{}\n'.format(99,1,1,1))
    colour_map.close()
    print("colour map saved as:",obj_path_dir+'/loop_colour_map.clr')

obj_path_dir= 'C:/Users/mark/Desktop/model2_m'   # directory of existing obj surfaces
colour_path='C:/Users/mark/Desktop/model2_m'
geoh5_create_surface_data(obj_path_dir,colour_path)


### dxf format surfaces from directory of obj files

In [ ]:
#code to parse a directory of .obj files and combine them into
#a single 3D DXF file that Micromine, Datamine, Vulcan... can read
#(note .dxf meshes are 1 based)
#Run this code after obj files have been created (see Example notebooks)

import pandas as pd
from os import listdir
from os.path import isfile, join
import numpy as np

def dxf_create_surface_data(obj_path_dir):


    onlyfiles = [f for f in listdir(obj_path_dir) if isfile(join(obj_path_dir, f))]
    
    fout = open(obj_path_dir+'/loop.dxf', "w")
    fout.write("  0\nSECTION\n  2\nENTITIES\n")

    for file in onlyfiles:
        if ('.obj' in file):
            obj=pd.read_csv(obj_path_dir+'/'+file,' ',names=["code","X","Y","Z"])
            indices=obj[obj['code']=='f']
            vertices=obj[obj['code']=='v']
            vertices=vertices.drop(['code'], axis=1)
            indices=indices[list("XYZ")].astype(int)
            i=indices.to_numpy()-1
            v=vertices.to_numpy()
           
            if(len(i)>0 and len(v)>0):
                for ind in range(0, i.shape[0]):
                    triangle="  0\n3DFACE\n  8\n{}\n  10\n{}\n  20\n{}\n  30\n{}\n  11\n{}\n  21\n{}\n  31\n{}\n  12\n{}\n  22\n{}\n  32\n{}\n  13\n{}\n  23\n{}\n  33\n{}\n".format(
                                file.replace('.obj',''),v[i[ind,0],0], v[i[ind,0],1], v[i[ind,0],2]
                                                        ,v[i[ind,1],0], v[i[ind,1],1], v[i[ind,1],2]
                                                        ,v[i[ind,2],0], v[i[ind,2],1], v[i[ind,2],2]
                                                        ,v[i[ind,2],0], v[i[ind,2],1], v[i[ind,2],2])
                    fout.write(triangle)
  
    fout.write("0\nENDSEC\n  0\nEOF\n")
    fout.write("END\n")
    fout.close()

            
#obj_path_dir= './Example2/vtkleaflet_2021-07-16-09-43/'   # directory of existing obj surfaces
dxf_create_surface_data(obj_path_dir)

### Gocad ts format surfaces from directory of obj files

In [ ]:
#code to parse a directory of .obj files and combine them into
#a set of Gocad .ts files that GeoscienceAnalyst and Gocad can read
#(note .ts meshes are 1 based)
#Run this code after obj files have been created (see Example notebooks)

import pandas as pd
from os import listdir
from os.path import isfile, join
import numpy as np

def gocad_create_surface_data(obj_path_dir):


    onlyfiles = [f for f in listdir(obj_path_dir) if isfile(join(obj_path_dir, f))]
    
    for file in onlyfiles:
        if ('.obj' in file):
            obj=pd.read_csv(obj_path_dir+'/'+file,' ',names=["code","X","Y","Z"])
            indices=obj[obj['code']=='f']
            vertices=obj[obj['code']=='v']
            vertices=vertices.drop(['code'], axis=1)
            indices=indices[list("XYZ")].astype(int)
            i=indices.to_numpy()
            v=vertices.to_numpy()
            
            if(len(i)>0 and len(v)>0): 
                fout = open(obj_path_dir+'/'+file.replace('.obj','')+'.ts', "w")
                #fout.write("GOCAD Tsurf 1 \nHEADER {\nname:" + file.replace('.obj','') + "\n}\nTFACE\n")
                fout.write("GOCAD TSurf 1\nHEADER {\nname:"+
                        file.replace('.obj','')+
                        "\n}\nTFACE\n")

                for ind in range(0, v.shape[0]):
                    fout.write("PVRTX %d %f %f %f\n" % (ind + 1, v[ind,0], v[ind,1], v[ind,2]))

                for ind in range(0,i.shape[0]):
                    fout.write("TRGL %d %d %d\n" % (i[ind,0], i[ind,1], i[ind,2]))


                fout.write("END\n")
                fout.close()

            
#obj_path_dir= './Example2/vtkleaflet_2021-07-16-09-43/'   # directory of existing obj surfaces
gocad_create_surface_data(obj_path_dir)

### Open Mining Format omf format surfaces from directory of obj files

In [ ]:
#code to parse a directory of .obj files and combine them into
#a single 3D OMF file that Micromine, Datamine, Vulcan... can read
#(note .omf meshes are 1 based) 
# requires pip install omf (and omfvista and  ipyvtklink if you want to visualise them)
# https://omf.readthedocs.io/en/latest/content/examples.html
# https://gmggroup.org/projects/data-exchange-for-mine-software/
# https://github.com/gmggroup/omf
#Run this code after obj files have been created (see Example notebooks)

from os import listdir
from os.path import isfile, join
import numpy as np
import omf
import random
import pandas as pd
import copy
def hextoints(h):
    '''Takes a hex rgb string (e.g. #ffffff) and returns an RGB tuple (float, float, float).'''
    return tuple(int(h[i:i + 2], 16) for i in (1, 3, 5))

def omf_create_surface_data(obj_path_dir,colour_path):

    proj = omf.Project(
        name='Loop project',
        description='Bunch of surfaces'
    )

    onlyfiles = [f for f in listdir(obj_path_dir) if isfile(join(obj_path_dir, f))]

    all_sorts = pd.read_csv(os.path.join(colour_path, 'all_sorts_clean.csv'), ",")
    all_sorts=all_sorts.set_index('code')
    colour_map=open(obj_path_dir+'/loop_colour_map.clr','w')
    colour_map.write('{\tStart\tRed\tGreen\tBlue\t}\n')
    
    surfaces=[]
    for file in onlyfiles:
        if ('.obj' in file):

            obj=pd.read_csv(obj_path_dir+'/'+file,' ',names=["code","X","Y","Z"])
            indices=obj[obj['code']=='f']
            vertices=obj[obj['code']=='v']
            vertices=vertices.drop(['code'], axis=1)
            indices=indices[list("XYZ")].astype(int)
            i=indices.to_numpy()-1
            v=vertices.to_numpy()
            i=np.ascontiguousarray(i)
            v=np.ascontiguousarray(v)
            
            if('Fault_' in file or 'dtm' in file):
                rgb=255,255,255
            else:
                rgb=hextoints(all_sorts.loc[file.replace('.obj','')]['colour'])

            if(len(i)>0 and len(v)>0):
                surf = omf.SurfaceElement(
                    name=file.replace('.obj',''),
                    geometry=omf.SurfaceGeometry(
                        vertices=v,
                        triangles=i
                    ),
                    data=[
                        omf.ScalarData(
                            name='rand vert data',
                            array=np.random.rand(v.shape[0]),
                            location='vertices'
                        ),
                        omf.ScalarData(
                            name='rand face data',
                            array=np.random.rand(i.shape[0]),
                            location='faces'
                        )
                        ],           
                    color=[rgb[0],rgb[1],rgb[2]]
                    )
                surfaces.append(copy.deepcopy(surf))

    proj.elements = surfaces

    assert proj.validate()
    
    omf.OMFWriter(proj, obj_path_dir+'/loop_surf.omf')

            
obj_path_dir= './Example2/vtkleaflet_2021-07-16-09-43/'   # directory of existing obj surfaces
colour_path='./Example2/tmp/'
omf_create_surface_data(obj_path_dir,colour_path)


### GeoscienceAnalyst geohy5 format voxel model from LoopStructural  model

In [ ]:
#code to take a LoopStructural voxel model and save it out
#as a *.geoh5 GeoscienceAnalyst model 
#weird indexing because default LS block has X & Y swapped & Z -ve
#assumes model already created by LoopStructural, and minx, maxx info from main calculations
#Requires installation of https://github.com/MiraGeoscience/geoh5py 

# assumes LoopStructural model object has been calculated

from pathlib import Path
import numpy as np
from geoh5py.objects import BlockModel
from geoh5py.workspace import Workspace
 
voxel_size=500
sizex=int((maxx-minx)/voxel_size)
sizey=int((maxy-miny)/voxel_size)
sizez=int((model_top-model_base)/voxel_size)
voxels=model.evaluate_model(model.regular_grid(nsteps=(sizey,sizex,sizez),shuffle=False),scale=False)

def create_geoh5_block_model_data(voxels,voxel_size,minx,miny,maxx,maxy,model_base,model_top,output_dir):

    name = "MyLoopBlockModel"

    # Generate a 3D array

    nodal_y = np.arange(0,maxx-minx+1,voxel_size)
    nodal_x = np.arange(0,maxy-miny+1,voxel_size)
    nodal_z = np.arange(model_top-model_base+1,0,-voxel_size)

    h5file_path = output_dir+"/loop_block_model.geoh5"


    # Create a workspace
    workspace = Workspace(h5file_path)

    grid = BlockModel.create(
        workspace,
        origin=[minx+(voxel_size/2), miny+(voxel_size/2), model_base+(voxel_size/2)],
        u_cell_delimiters=nodal_x,
        v_cell_delimiters=nodal_y,
        z_cell_delimiters=nodal_z,
        name=name,
        rotation=0,
        allow_move=False,
    )
    data = grid.add_data(
        {
            "DataValues": {
                "association": "CELL",
                "values": (
                    voxels.reshape((nodal_x.shape[0]-1,nodal_y.shape[0]-1,nodal_z.shape[0]-1)).transpose((1,0,2))
                ),
            }
        }
    )
    workspace.save_entity(grid)
    workspace.finalize()

output_dir= ''   # output directory to save geoh5 format voxel mdoel
create_geoh5_block_model_data(voxels,voxel_size,minx,miny,maxx,maxy,model_base,model_top,output_dir)



### Open Mining Format omf format voxel model from LoopStructural  model

In [ ]:
#code to take a LoopStructural voxel model and save it out
#as a *.omf file that Micromine, Datamine, Vulcan... can read
#weird indexing because default LS block has X & Y swapped & Z -ve
# requires pip install omf (and omfvista and  ipyvtklink if you want to visualise them)
# https://omf.readthedocs.io/en/latest/content/examples.html
# https://gmggroup.org/projects/data-exchange-for-mine-software/
# https://github.com/gmggroup/omf
#assumes model already created by LoopStructural, and minx, maxx info from main calculations

# assumes LoopStructural model object has been calculated

voxel_size=500
sizex=int((maxx-minx)/voxel_size)
sizey=int((maxy-miny)/voxel_size)
sizez=int((model_top-model_base)/voxel_size)
voxels=model.evaluate_model(model.regular_grid(nsteps=(sizey,sizex,sizez),shuffle=False),scale=False)

import omf
def create_omf_block_model_data(voxels,voxel_size,minx,miny,maxx,maxy,model_base,model_top,output_dir,colour_path):

    name = "MyLoopBlockModel"

    # Generate a 3D array
    proj = omf.Project(
        name='Loop project',
        description='Loop Block Model'
    )

    nodal_x = int((maxx-minx+1)/voxel_size)
    nodal_y = int((maxy-miny+1)/voxel_size)
    nodal_z = int((model_top-model_base+1)/voxel_size)
    print(nodal_x,nodal_y,nodal_z)
    print(minx,maxx,miny,maxy,model_base,model_top,voxel_size)
    vol = omf.VolumeElement(
        name='vol',
        geometry=omf.VolumeGridGeometry(
            axis_u=(1,0,0),
            axis_v=(0,1,0),
            axis_w=(0,0,-1),
            tensor_u=np.ones(nodal_y).astype(float)*voxel_size*(nodal_x/nodal_y),
            tensor_v=np.ones(nodal_x).astype(float)*voxel_size*(nodal_y/nodal_x),
            tensor_w=np.ones(nodal_z).astype(float)*voxel_size,
            origin=[minx-(voxel_size/2),miny-(voxel_size/2),model_base-(voxel_size/2)]
        ),
        data=[
            omf.ScalarData(
                name='LithoData',
                location='cells',
                array=voxels.reshape((nodal_z,nodal_x,nodal_y)).transpose((0,1,2)).flatten()
            )
        ]
    )

    proj.elements = [vol]

    assert proj.validate()
    
    omf.OMFWriter(proj, output_dir+'/loop_block.omf')

output_dir= ''   # output directory to save omf format voxel mdoel
colour_path='./Example2/tmp/'

create_omf_block_model_data(voxels,voxel_size,minx,miny,maxx,maxy,model_base,model_top,output_dir,colour_path)